# Confusion Map generation
This is a tool to evaluate how well a trained model is doing.

It generates a map where:
True Negatives are marked Black
True Positives are marked Green
False Negatives are marked Blue
False Positives are marked Red

It also produces an Excel file with a confusion matrix.

 ## if evaluating a single class model

**Ignore this if you're evaluating a multiclass model**

Note that if evaluating a single class model:
 1. you need to set *output_classes=2* in the next cell
 1. also in the next cell specify the *single_class_species*. Default is 1 (Arundo) but refer to table below.
 1. Selecting the species from the user interface will be disabled & set to 1. This is normal.
A single class model only marks a 1 wherever the species it has been trained on is detected.  It doesn't know what that 1 refers to, so you have to specify what it (the single_class_species) is manually.  Failing to do so will fail to produce a correct confusion map. 

```
 1 = Arundo
 2 = Opuntia
 3 =  Agricultural area  (not a species)
 4 = Eucalyptus
 5 = Agave
 6= Acacia
 7 = Prinjol(Pines)  (not included in training during the project so it's intended only for future models)
```

In [1]:
# how many classes the model can output
# 1 for background + number of species/classes detectable.
# Multiclass models coming out of the project had  output_classes=7 
output_classes=2
single_class_species= 5    # this is ignored in multi-class mode

In [2]:
# only change this if you have trained for more classes than listed.

species_available=[1,2,4,5,6,7]

In [3]:

import os
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,50).__str__()
from pathlib import Path
import ipywidgets as widgets
import torch
import segmentation_models_pytorch as smp
import visualisations2 as vis2
import gc
from torch import cuda


In [4]:
model =smp.UnetPlusPlus(
    encoder_name="resnet152",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use None or `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=output_classes,                      # model output channels (number of classes in your dataset, add +1 for background)
    activation='softmax',  #deprecated for some models.  Last activation is self(x)
)

In [5]:


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Net=model.to(device)

In [6]:
Model=''
ModelPath=''
epochs=[]
epochNums=[]
speciesList=[]
saveString=''

disable_species_selection=False
if output_classes <= 2:  # in case of a single-class model
    output_classes = 2
    vis2.single_class_mode = True  # if false, detection operates on Multiclass
    vis2.single_class =single_class_species   # if operating on single class this is the class being detected
    species_available=[single_class_species]
    disable_species_selection=True

def create_dir(path):
	""" To creating a directory if it does not exist"""
	if not os.path.exists(path): os.makedirs(path)


def FinalOptions():
    global saveString
    global AcceptEpochsButton
    AcceptEpochsButton.close()
    
    saveString='./Results/'+Model+'/'
    create_dir(saveString)
    global species_available
    global disable_species_selection
    Speciesbox = widgets.SelectMultiple(options=species_available,
        value=[species_available[0]],
        rows=6,
        description='Choose species to detect:',
        disabled=disable_species_selection)
    
    ListRasters = os.listdir(".\\Data\\source\\labels")
    for i in ListRasters:
        if not os.path.isfile(".\\Data\\source\\labels\\"+i):
            ListRasters.remove(i)
    Rasterbox = widgets.SelectMultiple(
        options=ListRasters,
        value=[ListRasters[0]],
        rows=12,
        description='Select Rasters:',
        disabled=False)
    Tilebox = widgets.IntText(value=1600, description='Tile Size:', disabled=False)
    
    Execute = widgets.Button(description='Process', disabled=False,button_style='')
        
    def ProduceResults(b):
        global speciesList
        global Model
        global ModelPath
        global epochs
        global epochNums
        global speciesList
        global saveString
        
        speciesList=Speciesbox.value
        rasters=Rasterbox.value
        tile_size=Tilebox.value
        print("Rasters:", rasters, "  Species:",speciesList, "Tile Size=",tile_size)
        Speciesbox.close()
        Rasterbox.close()
        Tilebox.close()
        Execute.close()
        
        for i in range(len(epochNums)):
    
            epoch=epochs[i]
            epoch_num=epochNums[i]
            Net.load_state_dict(torch.load(epoch))
            for raster in rasters:
                image_file_path=".\\Data\\source\\rasters\\"+raster 
                label_file_path=".\\Data\\source\\labels\\"+raster 
                save_as=saveString+raster[:-4]+'-Ep'+str(epoch_num)
                RawPrediction, Lbl=vis2.GeneratePredictionWithLabel(Net, image_file_path,label_file_path, tilesize=tile_size, save_name=None)
                df,ArgmaxMap=vis2.Pred2Result(RawPrediction, Lbl, save_as)
                del RawPrediction
                gc.collect()
                cuda.empty_cache()
                #ArgmaxMap=ArgmaxMapOnly(Net, image_file_path, tilesize=1600, save_name=None)
                for s in speciesList:
                    vis2.Argmax2ConfMap(ArgmaxMap,Lbl,save_as,species=s)
                    print(s, "saved")
        
    
    Execute.on_click(ProduceResults)

    display(Rasterbox, Speciesbox, Tilebox, Execute)
    
    

def processModel():
    AcceptModel.close()
    global AcceptEpochsButton
    AcceptEpochsButton = widgets.Button(description='Accept & Proceed', disabled=False,button_style='')
    ListEpochs = []
    for file in os.listdir(ModelPath):
        if file[-6:]=='.torch':
            ListEpochs.append(file)

    Epochsbox= widgets.SelectMultiple(options=ListEpochs,
        value=[ListEpochs[0]],
        rows=15,
        description='Epochs:',
        disabled=False)

    def AcceptEpochs(b):
        global epochs
        global epochNums
        global ModelPath
        global Model
        
        for i in range(len(Epochsbox.value)):
            epochs.append(ModelPath+Epochsbox.value[i])
        for i in range(len(epochs)):
            epochNums.append(epochs[i].split('/')[-1].split('-')[0])

        Epochsbox.close()
        Suggestionbox.close()
        row1.close()
        print("Epochs: ", epochs )
        AcceptEpochsButton.close()
        FinalOptions()
        return
 
    AcceptEpochsButton.on_click(AcceptEpochs)
    
    #===================================================
    log_path='.\\Models\\'+Model+'\\'+"Log for MC-"+Model+'.csv'
    best_epochs_suggestion='Suggested Epochs: '
    for i in vis2.FindBestEpochs(log_path):
        best_epochs_suggestion=best_epochs_suggestion+str(i)+', '
    best_epochs_suggestion=best_epochs_suggestion[0:-2]
    Suggestionbox=widgets.Label(best_epochs_suggestion)
    
    row1=widgets.HBox([Epochsbox,Suggestionbox])
    display(row1, AcceptEpochsButton)



ListModels = []
for file in os.listdir(".\\Models\\"):
    d = os.path.join(".\\Models\\", file)
    if os.path.isdir(d):
        ListModels.append(file)
if '_usable' in ListModels: ListModels.remove('_usable')

Modelsbox = widgets.Select(
    options=ListModels,
    value=ListModels[0],
    rows=10,
    description='Pick Model:',
    disabled=False)

AcceptModel = widgets.Button(description='Accept & Proceed', disabled=False,button_style='')

def GetModel(b):
    global Model
    global ModelPath
    Model=Modelsbox.value
    ModelPath='./Models/'+ Model +'/'
    Modelsbox.close()
    print("Model: ", Model )
    processModel()
    return
 
        
AcceptModel.on_click(GetModel)

display(Modelsbox, AcceptModel)




Select(description='Pick Model:', options=('UnetPlusPlus-ResNetEnc-1048-12-7-2023',), rows=10, value='UnetPlus…

Button(description='Accept & Proceed', style=ButtonStyle())

In [ ]:
vis2.single_class_mode

In [7]:
vis2.single_class_species


1